Deep learning is a great tool that helps us efficiently summarize inherent patterns from tons of input data. I'd like to introduce DeepLearning.scala by letting the framework learn the common difference from Arithmetic progression.

## Background


**Input**:
 Arithmetic progression(AP) as:
``` val input: INDArray = Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray``` 


**Output**: 
 Common Difference of the certain AP as: 
```val expectedOutput: INDArray = Array(Array(1), Array(3), Array(2)).toNDArray```

So here we want DeepLearning.scala to learn the common difference from the AP, i.e. ```{1} from {0, 1, 2} ``` 
in which `2-1 = 1-0 = 1 `

## Install DeepLearning.scala

DeepLearning.scala is hosted on Maven Central repository.

You can use magic imports in [jupyter-scala](https://github.com/alexarchambault/jupyter-scala) or [Ammonite-REPL](http://www.lihaoyi.com/Ammonite/#Ammonite-REPL) to download DeepLearning.scala and its dependencies.

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0`

import $ivy.`org.nd4j:nd4j-native-platform:0.7.2`

import $plugin.$                                                                             


import $ivy.$                                                       

import $ivy.$                                                           

import $ivy.$                                                       

import $ivy.$                                                          

import $ivy.$                                                         

import $ivy.$                                                         

import $ivy.$                                                             

import $ivy.$                                                            


import $ivy.$                                    

If you use [sbt](http://www.scala-sbt.org), please add the following settings into your `build.sbt`:

```
libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableany" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablenothing" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableseq" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiabledouble" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablefloat" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablehlist" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablecoproduct" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableindarray" % "latest.release"

addCompilerPlugin("com.thoughtworks.implicit-dependent-type" %% "implicit-dependent-type" % "latest.release")

addCompilerPlugin("org.scalamacros" % "paradise" % "2.1.0" cross CrossVersion.full)

fork := true

scalaVersion := "2.11.8"
```

Note that this example must run on Scala 2.11.8 or Scala 2.10.6 because [nd4s](http://nd4j.org/scala) does not support Scala 2.12. Make sure there is not a setting like `scalaVersion := "2.12.1"` in your `build.sbt`.

See [Scaladex](https://index.scala-lang.org/thoughtworksinc/deeplearning.scala) to install DeepLearning.scala in other build tools!

Then, you may want to import classes in DeepLearning.scala and its dependencies.

In [2]:
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.DifferentiableHList
import com.thoughtworks.deeplearning.DifferentiableINDArray
import com.thoughtworks.deeplearning.Layer
import com.thoughtworks.deeplearning.Symbolic
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4s.Implicits._
import shapeless._

import com.thoughtworks.deeplearning.DifferentiableHList._

import com.thoughtworks.deeplearning.DifferentiableDouble._

import com.thoughtworks.deeplearning.DifferentiableINDArray._

import com.thoughtworks.deeplearning.DifferentiableAny._

import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._

import com.thoughtworks.deeplearning.Symbolic._

import com.thoughtworks.deeplearning.DifferentiableHList

import com.thoughtworks.deeplearning.DifferentiableINDArray

import com.thoughtworks.deeplearning.Layer

import com.thoughtworks.deeplearning.Symbolic

import com.thoughtworks.deeplearning.Poly.MathFunctions._

import com.thoughtworks.deeplearning.Poly.MathOps

import org.nd4j.linalg.api.ndarray.INDArray

import org.nd4j.linalg.factory.Nd4j

import org.nd4s.Implicits._

import shapeless._

## Design the neural network

DeepLearning.scala is also a language that we can use to create complex neural networks.

In the following sections, you will learn:
 * how to define types for a neural network
 * how to use a neural network as a predictor
 * how to create a neural network
 * how to train a neural network

### The type of neural networks

Like a `scala.Function`, a neural network has its own input types and output types.

For example, the type of the neural network that accepts an N-dimensional array and returns another N-dimensional array is `(INDArray => INDArray)@Symbolic`.

In [3]:
var myNeuralNetwork: (INDArray => INDArray)@Symbolic = _

myNeuralNetwork: (Symbolic.FromTo[INDArray, INDArray]{type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray;type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray})#@ = null

In `(A => B)@Symbolic`, A is the input type, and B is the output type. For the example above, both the input type and the output type are `INDArray`.


`@Symbolic` is a syntactic sugar to create implicit dependent types. See [implicit-dependent-type](https://github.com/ThoughtWorksInc/implicit-dependent-type) for more information.

### Use a neural network as a predictor

Like a normal `scala.Function`, if you pass the input data to the neural network, it will return some results.
You can use the `predict` method to invoke a neural network.

In [4]:
val input: INDArray = Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray
val predictionResult: INDArray = myNeuralNetwork.predict(input)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


: 

The above code throws a `NullPointerException` because `myNeuralNetwork` has not been initialized.
We will fix the problem by creating a valid neural network.

### Create a neural network

Same as the definition of a normal Scala function, the definition of neural network consists of a type definition for its parameter, a type definition for its return value, and a body that contains mathematical formulas, function-calls, and control flows.

#### Weight Intialization 

A neural network is trainable.
It means that some variables in the neural network can be changed automatically according to some goals. Those variables are called `weight`.
You can create weight variables via `toWeight` method, given its initial value.

In order to create a weight, you must create an `Optimizer`, which contains the rule that manages how the weight changes. See [Scaladoc](https://javadoc.io/page/com.thoughtworks.deeplearning/unidoc_2.11/latest/com/thoughtworks/deeplearning/DifferentiableINDArray$$Optimizers$.html) for a list of built-in optimizers.

In [5]:
implicit def optimizer: Optimizer = new LearningRate {
  def currentLearningRate() = 0.001
}

defined function optimizer

#### `@Symbolic` placeholders

In [6]:
def createMyNeuralNetwork(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val initialValueOfWeight: INDArray = Nd4j.randn(3, 1) / math.sqrt(3.0)
  val weight: INDArray @Symbolic = initialValueOfWeight.toWeight
  input dot weight
}

defined function createMyNeuralNetwork

The `createMyNeuralNetwork` method is a **symbolic method**.
When you call the `createMyNeuralNetwork` method, you have not actually evaluated it yet.
In fact, you only build its structure.
Variables in the neural network are placeholders,
which will be replaced with actual values in the future training or prediction process.

Generally, any method that contains an `implicit` parameter that has a `@Symbolic` type is a **symbolic method**.
You can create symbolic methods in the following form:

```
def yourSymbolicMethod(implicit input: InputType @Symbolic): OutputType @Symbolic = {
  ???
}
```

The `@Symbolic` annotations in symbolic methods create placeholder types for parameters, return values and other local variables.

Note that the input parameter must be `implicit` so that it is automatically generated when you invoke the the symbolic method. Otherwise, you have to manually pass the `@Symbolic` annotated placeholder to it.

Now we create a new neural network and assign it to `myNeuralNetwork`.

In [7]:
myNeuralNetwork = createMyNeuralNetwork

## Train your Neural Network

You have learned that weight will be automatically changed due to some goals.

In DeepLearning.scala, when we train a neural network, our goal should always be minimizing the absolute of the return value.

For example, if someone repeatedly call `myNeuralNetwork.train(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray)`,
the neural network would try to minimize `input dot weight`.
Soon `weight` would become an array of zeros in order to make `input dot weight` zeros,
and `myNeuralNetwork.predict(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray)` would return `Array(Array(0), Array(0), Array(0)).toNDArray`.

What if you expect `myNeuralNetwork.predict(Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray)` to return `Array(Array(1), Array(3), Array(2)).toNDArray`?

You can create another neural network that evaluates how far between the result of `myNeuralNetwork` and your expectation. The new neural network is usually called **loss function**.

In [8]:
def lossFunction(implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val expectedOutput = pair.tail.head
  abs(myNeuralNetwork.compose(input) - expectedOutput).sum
}

defined function lossFunction

When the `lossFunction` get trained continuously, its return value will be close to zero, and the result of  `myNeuralNetwork` must be close to the expected result at the same time.

Note the `lossFunction` accepts a placehold of `INDArray :: INDArray :: HNil` as its parameter, which is  a [shapeless](https://github.com/milessabin/shapeless)'s `HList` type.
The `HList` consists of two N-dimensional arrays.
The first array is the input data used to train the neural network, and the second array is the expected output.

Now, we hard-code some data to train the network:

In [9]:
val input: INDArray = Array(Array(0, 1, 2), Array(3, 6, 9), Array(13, 15, 17)).toNDArray
val expectedOutput: INDArray = Array(Array(1), Array(3), Array(2)).toNDArray

val NumberOfIterations = 400

val lossSeq = for (iteration <- 0 until NumberOfIterations) yield {
  lossFunction.train(input :: expectedOutput :: HNil)
}

input: INDArray = [[0.00, 1.00, 2.00],
 [3.00, 6.00, 9.00],
 [13.00, 15.00, 17.00]]
expectedOutput: INDArray = [1.00, 3.00, 2.00]
NumberOfIterations: Int = 400
lossSeq: collection.immutable.IndexedSeq[Symbolic.To.<refinement>.this.type.OutputData] = Vector(
  11.161251068115234,
  9.637250900268555,
  8.113250732421875,
  6.589250564575195,
  5.065250396728516,
  3.541250467300415,
  2.1384034156799316,
  2.5212504863739014,
  2.4424033164978027,
  2.242403507232666,
  2.042403221130371,
...

After those iterations, the loss should close to zero.

In [10]:
println(s"loss: ${ lossFunction.predict(input :: expectedOutput :: HNil) }")

loss: 1.1507331132888794


The prediction result should close to Array(Array(0), Array(3), Array(1)).toNDArray

In [11]:
println(s"result: ${ myNeuralNetwork.predict(input) }")

result: [1.00, 3.22, 2.93]


Then we create a plot showing the loss changes during iterations.

In [12]:
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import $ivy.$                                             

In [13]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._

In [14]:
plotly.JupyterScala.init()
val plot = Seq(
  Scatter(
   0 until NumberOfIterations,
   lossSeq
  )
)
plot.plot(
  title = "loss by time"
)

plot: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
          0.0,
          1.0,
          2.0,
          3.0,
          4.0,
          5.0,
          6.0,
...
res13_2: String = "plot-1668384299"

## Summary

In this article, you have learned:
* to create neural networks dealing with complex data structures like `Double`, `INDArray` and `HList` like ordinary programming language
* to compose a neural network into a larger neural network
* to train a neural network
* to use a neural network as a predictor